<a href="https://colab.research.google.com/github/seonghunYang/big_contest2020/blob/master/Model10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
vs_r_team = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/모델/한줄에 상대최근 꼴아박기.csv', encoding='utf-8-sig')
vs_r_team_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/2차_최종_최근_상대_팀타자.csv', encoding='utf-8-sig')
vs_r_team_pitcher = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/최근/2차_최종_최근_상대_팀투수.csv', encoding='utf-8-sig')

In [ ]:
vs_r_team[['팀코드', '상대팀코드', '득점권타율']]

,팀코드,상대팀코드,득점권타율
0,LG,HH,0.333
1,HH,LG,0.200
2,NC,HT,0.143
3,HT,NC,0.100
4,SK,KT,0.375
...,...,...,...
5627,SK,HH,0.400
5628,LT,WO,0.167
5629,NC,OB,0.250
5630,OB,NC,0.267


In [ ]:
# 게임 1경기 짜리 전부 제거
col = ['20160412NCSS0', '20160414OBHH0', '20160415LGHH0', '20160423HTLT0',
       '20160504OBLG0', '20160505HHSK0', '20160505LTHT0', '20160507HHKT0',
       '20160508LTOB0', '20160511SSLG0', '20160511WOLT0', '20160520OBLT0',
       '20160522KTHH0', '20160529LGOB0', '20160609WONC0', '20160622LTHT0',
       '20160719KTHH0', '20160810SSHH0', '20160824LGOB0', '20160901HTSS0',
       '20170402HTSS0', '20170411HTOB0', '20170427SSHT0', '20170504OBSS0',
       '20170525SKLT0', '20170526SSWO0', '20170608LTNC0', '20170611SKLG0',
       '20170618LGHT0', '20170621OBHT0', '20170629SSHT0', '20170708HTKT0',
       '20170722HHOB0', '20170910WOSK0', '20170916KTHT0', '20170917OBSS0',
       '20171001HTKT0', '20180327SSHT0', '20180331OBKT0', '20180403LTHH0',
       '20180520NCKT0', '20180622OBSS0', '20180623LTLG0', '20180711WOHH0',
       '20180721OBLG0', '20180727HHOB0', '20180805WOKT0', '20180809WOHH0',
       '20180811HTSK0', '20180812HTSK0', '20180911OBLT0', '20180918HTSS0',
       '20180919SKKT0', '20180925LGSK0', '20181003HTSS0', '20190327SSLT0',
       '20190407HHLT0', '20190421SSHH0', '20190508HHSK0', '20190526KTHT0',
       '20190806LGHT0', '20190807WOLT0', '20190825WOSS0', '20200531NCSS0',
       '20200613WONC0', '20200619OBLG0', '20200625NCKT2']
for key in col:
  vs_r_team_hitter = vs_r_team_hitter[vs_r_team_hitter['게임키'] != key]
  vs_r_team_pitcher = vs_r_team_pitcher[vs_r_team_pitcher['게임키'] != key]

In [ ]:
vs_r_team_hitter = vs_r_team_hitter.reset_index()
vs_r_team_pitcher = vs_r_team_pitcher.reset_index()
del vs_r_team_hitter['index']
del vs_r_team_pitcher['index']

In [ ]:
vs_r_team_hitter = vs_r_team_hitter[['게임키', '일자', '팀코드', '상대팀코드', '초말', '타자', '타수', '타점', '득점',
       '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비', '4구', '고4', '사구',
       '삼진', '병살타', '실책', '잔루', '득점권타율', '득점권타수', '득점권안타', '시즌', '단타',]]

In [ ]:
vs_r_team_pitcher = vs_r_team_pitcher[['게임키', '일자', '팀코드', '상대팀코드', '초말', '완투', '결과', '홀드', '이닝*3',
       '투구수', '타자', '타수', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비',
       '4구', '고4', '사구', '삼진', '병살타', '폭투', '보크', '실책', '실점', '자책점', '득점권WHIP',
       '2점차이하WHIP', '345번타자WHIP',  '시즌']]

In [ ]:
r_team = pd.merge(vs_r_team_hitter, vs_r_team_pitcher, on=['게임키', '일자', '팀코드', '상대팀코드', '초말', '시즌'])

In [ ]:
# 2020년 데이터 뺴기
r_team = r_team[r_team['시즌'] != 2020]
vs_r_team = vs_r_team[vs_r_team['시즌'] != 2020]

In [ ]:
lstm_inputs = []
for idx in range(0, len(vs_r_team), 1):
  row = vs_r_team.iloc[idx]
  previous = vs_r_team[(vs_r_team['일자'] < row['일자']) & (vs_r_team['팀코드'] == row['팀코드'] ) & (vs_r_team['상대팀코드'] == row['상대팀코드']) & (vs_r_team['시즌'] == row['시즌'])].tail(10)
  if len(previous) < 10:
    continue
  if len(lstm_inputs) == 0:
    lstm_inputs = previous
  else:
    lstm_inputs = pd.concat([lstm_inputs, previous])
lstm_inputs

,2017,2018,2019,2020,2루타_x,3루타_x,4구_x,HH,HT,KT,LG,LT,OB,SK,SS,WO,게임키,고4_x,단타_x,더블헤더코드,도루_x,도루실패_x,득점,득점권안타,득점권타수,득점권타율,병살타_x,사구_x,삼진_x,상대최근AB,상대최근B1B,상대최근B2B,상대최근B3B,상대최근BABIP,상대최근BB,상대최근BB%_x,상대최근BB/K,상대최근CS,상대최근GDP,상대최근GPA,...,2경기전보크,2경기전병살타_y,2경기전4구_y,3경기전득점권안타,3경기전홈런_x,3경기전희비_x,3경기전4구_x,3경기전안타_x,3경기전잔루,3경기전사구_x,3경기전병살타_x,3경기전도루실패_x,3경기전타자_x,3경기전득점권타수,3경기전초말,3경기전삼진_x,3경기전3루타_x,3경기전고4_x,3경기전실책_x,3경기전도루_x,3경기전자책점,3경기전결과,3경기전투구수,3경기전실책_y,3경기전득점권WHIP,3경기전단타_y,3경기전홈런_y,3경기전2루타_y,3경기전희비_y,3경기전이닝*3,3경기전3루타_y,3경기전사구_y,3경기전도루실패_y,3경기전홀드,3경기전폭투,3경기전2점차이하WHIP,3경기전삼진_y,3경기전보크,3경기전병살타_y,3경기전4구_y
28,0.0,0.0,0.0,0.0,0,0,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,20160405SKLT0,0,5,0,0,0,2,1,9,0.111,0,1,7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0
36,0.0,0.0,0.0,0.0,3,0,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,20160406SKLT0,0,8,0,1,1,11,5,10,0.500,0,1,2,30.000000,5.000000,0.000000,0.000000,0.217391,3.000000,0.088235,0.428571,0.000000,0.000000,0.160784,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0
46,0.0,0.0,0.0,0.0,1,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,20160407SKLT0,0,3,0,0,0,3,0,3,0.000,0,0,8,28.500000,6.500000,1.500000,0.000000,0.355556,2.500000,0.076923,0.555556,0.500000,0.000000,0.318777,...,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0
339,0.0,0.0,0.0,0.0,1,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,20160517LTSK0,0,10,0,0,0,3,2,10,0.200,1,0,8,29.333333,5.333333,1.333333,0.000000,0.294118,2.000000,0.061224,0.352941,0.333333,0.000000,0.262090,...,0,1,4,1,0,0,3,5,8,1,0,0,34,9,1,7,0,0,0,0,1,1,152,1,0.750,2,0,2,1,27,0,2,0,3,0,0.667,12,0,0,2
349,0.0,0.0,0.0,0.0,1,1,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,20160518LTSK0,0,5,0,1,1,3,2,11,0.182,0,0,8,31.333333,7.000000,1.666667,0.000000,0.351351,1.333333,0.039216,0.222222,0.333333,0.333333,0.280941,...,0,0,3,5,3,0,2,14,5,1,0,1,31,10,1,2,0,0,1,1,0,1,74,0,1.500,1,0,0,0,15,0,1,0,0,0,1.333,2,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5063,0.0,0.0,1.0,0.0,2,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,20190706LTWO0,0,9,0,1,0,6,3,9,0.333,1,0,8,35.000000,7.000000,1.666667,0.000000,0.366197,4.333333,0.109244,0.483871,0.000000,1.000000,0.269736,...,0,0,1,0,1,0,4,6,7,0,2,0,35,5,0,8,0,0,2,0,3,0,133,0,1.000,9,0,2,0,24,0,0,0,0,0,2.100,6,0,0,3
5073,0.0,0.0,1.0,0.0,2,2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,20190707LTWO0,0,4,0,0,0,5,3,9,0.333,2,1,8,35.333333,8.333333,2.333333,0.000000,0.438356,3.666667,0.091667,0.419355,0.000000,0.666667,0.290849,...,0,2,2,0,1,0,5,8,12,0,0,0,40,7,0,16,0,1,0,0,2,0,128,0,6.000,5,0,0,0,24,0,0,1,0,0,0.750,9,0,0,1
5413,0.0,0.0,1.0,0.0,1,0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,20190829LTWO0,0,7,0,2,0,4,3,12,0.250,1,0,8,33.333333,7.333333,3.000000,0.666667,0.428571,3.333333,0.086957,0.478261,0.000000,1.333333,0.310526,...,0,0,1,8,1,0,6,15,8,1,1,0,46,22,1,7,0,1,0,1,1,1,138,0,1.286,7,0,3,0,27,1,0,0,0,0,0.000,7,0,2,2
5423,0.0,0.0,1.0,0.0,1,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,20190830LTWO0,0,9,0,1,1,5,4,11,0.364,0,0,7,31.000000,6.666667,1.666667,0.666667,0.385714,3.000000,0.084906,0.375000,0.000000,1.333333,0.256870,...,0,1,4,3,0,0,2,11,6,0,1,0,36,9,1,8,0,0,1,1,2,1,140,0,2.400,9,0,1,0,27,0,0,0,2,0,0.783,9,0,0,1


In [ ]:
def matchup_numbering(df):
    match_up = df.sort_values(by=['시즌','팀코드','일자','상대팀코드'])
    match_up = match_up.reset_index(drop = True)
    matchup_num = 1
    keys = []
    num = []
    for i, row in match_up.iterrows():
        if i==0:
            season = match_up.at[i,'시즌']
            code = match_up.at[i,'팀코드']
            now_matchup = match_up.at[i,'상대팀코드']
            matchup_key = str(season)+str(code)+str(now_matchup)+str(matchup_num)
        num.append(matchup_num)
        keys.append(matchup_key)
        if i == match_up.index[-1]:
            break
        season = match_up.at[i,'시즌']
        next_season = match_up.at[i+1,'시즌']
        code = match_up.at[i,'팀코드']
        next_code = match_up.at[i+1,'팀코드']
        now_matchup = match_up.at[i,'상대팀코드']
        next_matchup = match_up.at[i+1,'상대팀코드']
        if season == next_season:
            if code == next_code:
                if now_matchup != next_matchup:
                    matchup_num=matchup_num+1
                    matchup_key = str(next_season)+str(next_code)+str(next_matchup)+str(matchup_num)
            else:
                matchup_num = 1
                matchup_key = str(next_season)+str(next_code)+str(next_matchup)+str(matchup_num)
        else:
            matchup_num = 1
            matchup_key = str(next_season)+str(next_code)+str(next_matchup)+str(matchup_num)
    match_up['매치업'] = num
    match_up['매치업키'] = keys
    return match_up.sort_values(by=['시즌','매치업'])

In [ ]:
vs_r_team_match = matchup_numbering(vs_r_team)
vs_r_team_match

,2017,2018,2019,2020,2루타_x,3루타_x,4구_x,HH,HT,KT,LG,LT,OB,SK,SS,WO,게임키,고4_x,단타_x,더블헤더코드,도루_x,도루실패_x,득점,득점권안타,득점권타수,득점권타율,병살타_x,사구_x,삼진_x,상대최근AB,상대최근B1B,상대최근B2B,상대최근B3B,상대최근BABIP,상대최근BB,상대최근BB%_x,상대최근BB/K,상대최근CS,상대최근GDP,상대최근GPA,...,2경기전4구_y,3경기전득점권안타,3경기전홈런_x,3경기전희비_x,3경기전4구_x,3경기전안타_x,3경기전잔루,3경기전사구_x,3경기전병살타_x,3경기전도루실패_x,3경기전타자_x,3경기전득점권타수,3경기전초말,3경기전삼진_x,3경기전3루타_x,3경기전고4_x,3경기전실책_x,3경기전도루_x,3경기전자책점,3경기전결과,3경기전투구수,3경기전실책_y,3경기전득점권WHIP,3경기전단타_y,3경기전홈런_y,3경기전2루타_y,3경기전희비_y,3경기전이닝*3,3경기전3루타_y,3경기전사구_y,3경기전도루실패_y,3경기전홀드,3경기전폭투,3경기전2점차이하WHIP,3경기전삼진_y,3경기전보크,3경기전병살타_y,3경기전4구_y,매치업,매치업키
0,0.0,0.0,0.0,0.0,2,0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20160401HHLG0,0,11,0,0,0,4,3,15,0.200,1,0,10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,1,2016HHLG1
1,0.0,0.0,0.0,0.0,4,0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20160402HHLG0,1,9,0,0,1,7,5,15,0.333,0,2,9,46.000000,11.000000,2.000000,0.000000,0.361111,3.000000,0.057692,0.300000,0.000000,1.000000,0.228461,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,1,2016HHLG1
137,0.0,0.0,0.0,0.0,3,0,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20160401HTNC0,0,4,0,0,0,4,1,10,0.100,1,1,10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,1,2016HTNC1
138,0.0,0.0,0.0,0.0,2,0,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20160402HTNC0,0,5,0,1,1,4,3,7,0.429,0,0,8,34.000000,4.000000,3.000000,0.000000,0.304348,3.000000,0.078947,0.300000,0.000000,1.000000,0.245046,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,1,2016HTNC1
277,0.0,0.0,0.0,0.0,0,0,4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,20160401KTSK0,0,9,0,2,1,8,2,7,0.286,0,1,7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,1,2016KTSK1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4774,0.0,0.0,1.0,0.0,1,0,5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,20190928LGHT0,0,4,0,1,0,1,0,8,0.000,2,3,7,35.666667,9.666667,1.666667,0.333333,0.402299,3.666667,0.089431,0.523810,0.333333,0.333333,0.290403,...,2,9,0,1,7,14,12,3,0,0,51,23,1,9,1,0,2,2,2,1,147,0,1.000,6,0,1,0,27,0,0,0,0,0,0.000,5,0,0,4,65,2019LGHT65
5205,0.0,0.0,1.0,0.0,1,1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,20191001NCOB0,0,13,0,0,0,6,4,15,0.267,0,0,6,36.666667,5.000000,0.333333,0.333333,0.184783,1.333333,0.034783,0.266667,0.000000,0.000000,0.179644,...,2,0,2,0,0,4,2,0,0,0,31,0,0,3,0,0,0,0,3,0,109,0,3.000,5,1,2,0,24,0,0,0,0,0,1.304,2,0,1,2,65,2019OBNC65
5489,0.0,0.0,1.0,0.0,1,0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20190929SSKT0,0,5,0,0,0,0,0,7,0.000,0,0,10,31.666667,4.666667,0.666667,0.333333,0.239437,4.000000,0.107143,0.571429,0.333333,0.000000,0.251077,...,6,2,0,1,2,5,5,0,0,1,32,6,1,5,0,0,0,2,2,1,125,0,1.800,5,0,2,0,27,0,0,0,0,0,0.600,10,0,1,1,65,2019SSKT65
4775,0.0,0.0,1.0,0.0,0,0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,20190929OBLG0,0,4,0,0,0,0,0,1,0.000,0,0,6,36.333333,8.000000,2.000000,0.333333,0.373494,2.000000,0.051282,0.260870,0.333333,0.666667,0.285199,...,4,0,1,0,2,8,8,0,1,0,34,7,1,6,0,0,1,0,1,1,152,0,0.429,7,0,1,1,27,0,0,1,3,0,1.222,5,0,0,3,66,2019LGOB66


In [ ]:
vs_r_team_match = vs_r_team_match.reset_index()
del vs_r_team_match['index']

In [ ]:
idx_list = []
for idx in range(0, len(vs_r_team_match), 1):
  row = vs_r_team_match.iloc[idx]
  previous = vs_r_team_match[(vs_r_team_match['일자'] < row['일자']) & (vs_r_team_match['팀코드'] == row['팀코드'] ) & (vs_r_team_match['상대팀코드'] == row['상대팀코드']) & (vs_r_team_match['시즌'] == row['시즌'])].tail(10)
  if len(previous) < 10:
    continue
  idx_list.append(idx)

In [ ]:
decoder_outputs[['팀코드', '상대팀코드', '일자', '매치업키']].head(50)

,팀코드,상대팀코드,일자,매치업키
0,LT,SK,2016-06-18,2016LTSK23
1,LT,SK,2016-06-19,2016LTSK23
2,OB,KT,2016-06-23,2016OBKT24
3,SK,OB,2016-06-25,2016SKOB25
4,SK,OB,2016-06-26,2016SKOB25
5,HT,LG,2016-06-30,2016HTLG26
6,WO,HH,2016-06-29,2016WOHH26
7,WO,HH,2016-06-30,2016WOHH26
8,HH,SS,2016-07-09,2016HHSS29
9,HH,SS,2016-07-10,2016HHSS29


In [ ]:
decoder_outputs = vs_r_team_match.iloc[idx_list]
decoder_outputs = decoder_outputs[decoder_outputs['초말'] == 1]
decoder_outputs = decoder_outputs.reset_index()
del decoder_outputs['index'] 
decoder_outputs

,2017,2018,2019,2020,2루타_x,3루타_x,4구_x,HH,HT,KT,LG,LT,OB,SK,SS,WO,게임키,고4_x,단타_x,더블헤더코드,도루_x,도루실패_x,득점,득점권안타,득점권타수,득점권타율,병살타_x,사구_x,삼진_x,상대최근AB,상대최근B1B,상대최근B2B,상대최근B3B,상대최근BABIP,상대최근BB,상대최근BB%_x,상대최근BB/K,상대최근CS,상대최근GDP,상대최근GPA,...,2경기전4구_y,3경기전득점권안타,3경기전홈런_x,3경기전희비_x,3경기전4구_x,3경기전안타_x,3경기전잔루,3경기전사구_x,3경기전병살타_x,3경기전도루실패_x,3경기전타자_x,3경기전득점권타수,3경기전초말,3경기전삼진_x,3경기전3루타_x,3경기전고4_x,3경기전실책_x,3경기전도루_x,3경기전자책점,3경기전결과,3경기전투구수,3경기전실책_y,3경기전득점권WHIP,3경기전단타_y,3경기전홈런_y,3경기전2루타_y,3경기전희비_y,3경기전이닝*3,3경기전3루타_y,3경기전사구_y,3경기전도루실패_y,3경기전홀드,3경기전폭투,3경기전2점차이하WHIP,3경기전삼진_y,3경기전보크,3경기전병살타_y,3경기전4구_y,매치업,매치업키
0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,20160618SKLT0,0,8,0,0,0,0,0,4,0.000,2,0,4,31.666667,5.333333,1.000000,0.000000,0.256757,4.000000,0.109091,0.600000,0.000000,1.000000,0.216447,...,3,2,1,0,4,7,7,0,1,0,37,12,0,6,0,0,0,1,2,1,179,0,2.250,6,0,0,1,27,0,0,0,1,0,1.105,10,0,2,5,23,2016LTSK23
1,0.0,0.0,0.0,0.0,7,0,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,20160619SKLT0,0,3,0,1,0,10,5,14,0.357,1,2,5,32.000000,6.666667,0.333333,0.000000,0.269231,2.666667,0.075472,0.444444,0.000000,1.333333,0.196280,...,3,2,1,1,7,7,6,0,2,0,38,9,0,6,0,0,0,4,6,0,124,0,0.900,4,1,2,0,24,1,1,0,0,0,1.375,7,0,1,3,23,2016LTSK23
2,0.0,0.0,0.0,0.0,4,0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,20160623KTOB0,0,11,0,0,0,4,2,12,0.167,2,0,2,36.333333,9.000000,1.666667,0.000000,0.340426,2.666667,0.065041,0.571429,0.000000,0.333333,0.279741,...,4,2,0,1,2,8,5,0,0,0,36,6,0,3,0,0,0,0,4,0,136,1,1.000,10,1,0,1,24,0,0,1,0,1,1.800,9,0,1,0,24,2016OBKT24
3,0.0,0.0,0.0,0.0,2,0,5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,20160625OBSK0,1,6,0,0,0,6,3,12,0.250,0,0,8,32.000000,5.333333,1.000000,0.666667,0.276316,0.666667,0.019802,0.117647,0.666667,0.666667,0.240583,...,5,1,3,1,1,9,3,0,1,0,36,4,0,8,1,0,1,0,3,0,131,0,2.625,6,0,3,1,24,0,1,0,0,3,3.600,5,0,0,3,25,2016SKOB25
4,0.0,0.0,0.0,0.0,3,0,3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,20160626OBSK0,0,13,0,0,2,6,4,10,0.400,1,2,8,33.000000,6.000000,1.333333,0.333333,0.291139,1.666667,0.047170,0.352941,0.666667,0.333333,0.243630,...,1,0,0,0,0,7,4,0,1,2,31,1,0,3,0,0,1,0,4,0,152,2,1.091,6,1,2,1,24,0,0,0,0,1,3.000,5,0,0,5,25,2016SKOB25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011,0.0,0.0,1.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20190927SKSS0,0,4,0,3,0,0,0,5,0.000,1,0,7,29.000000,4.666667,1.000000,0.000000,0.236111,2.666667,0.080000,0.470588,1.000000,0.333333,0.176859,...,3,3,0,2,6,7,8,1,0,1,37,13,1,0,0,0,1,2,3,1,177,1,1.091,11,0,0,0,27,0,0,0,1,0,1.091,7,0,0,5,64,2019SSSK64
1012,0.0,0.0,1.0,0.0,5,0,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20190928SKSS0,1,6,0,1,0,9,5,15,0.333,0,0,10,30.000000,4.333333,0.333333,0.000000,0.212121,0.666667,0.021505,0.083333,0.666667,0.666667,0.119928,...,2,0,0,0,0,5,4,0,0,1,31,1,1,7,0,0,0,0,2,0,128,0,0.667,3,0,2,0,27,0,0,0,0,0,0.889,9,0,0,3,64,2019SSSK64
1013,0.0,0.0,1.0,0.0,1,1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,20191001NCOB0,0,13,0,0,0,6,4,15,0.267,0,0,6,36.666667,5.000000,0.333333,0.333333,0.184783,1.333333,0.034783,0.266667,0.000000,0.000000,0.179644,...,2,0,2,0,0,4,2,0,0,0,31,0,0,3,0,0,0,0,3,0,109,0,3.000,5,1,2,0,24,0,0,0,0,0,1.304,2,0,1,2,65,2019OBNC65
1014,0.0,0.0,1.0,0.0,0,0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,20190929OBLG0,0,4,0,0,0,0,0,1,0.000,0,0,6,36.333333,8.000000,2.000000,0.333333,0.373494,2.000000,0.051282,0.260870,0.333333,0.666667,0.285199,...,4,0,1,0,2,8,8,0,1,0,34,7,1,6,0,0,1,0,1,1,152,0,0.429,7,0,1,1,27,0,0,1,3,0,1.222,5,0,0,3,66,2019LGOB66


In [ ]:
decoder_outputs.groupby('매치업키').sum()

,2017,2018,2019,2020,2루타_x,3루타_x,4구_x,HH,HT,KT,LG,LT,OB,SK,SS,WO,고4_x,단타_x,더블헤더코드,도루_x,도루실패_x,득점,득점권안타,득점권타수,득점권타율,병살타_x,사구_x,삼진_x,상대최근AB,상대최근B1B,상대최근B2B,상대최근B3B,상대최근BABIP,상대최근BB,상대최근BB%_x,상대최근BB/K,상대최근CS,상대최근GDP,상대최근GPA,상대최근HBP,...,2경기전병살타_y,2경기전4구_y,3경기전득점권안타,3경기전홈런_x,3경기전희비_x,3경기전4구_x,3경기전안타_x,3경기전잔루,3경기전사구_x,3경기전병살타_x,3경기전도루실패_x,3경기전타자_x,3경기전득점권타수,3경기전초말,3경기전삼진_x,3경기전3루타_x,3경기전고4_x,3경기전실책_x,3경기전도루_x,3경기전자책점,3경기전결과,3경기전투구수,3경기전실책_y,3경기전득점권WHIP,3경기전단타_y,3경기전홈런_y,3경기전2루타_y,3경기전희비_y,3경기전이닝*3,3경기전3루타_y,3경기전사구_y,3경기전도루실패_y,3경기전홀드,3경기전폭투,3경기전2점차이하WHIP,3경기전삼진_y,3경기전보크,3경기전병살타_y,3경기전4구_y,매치업
매치업키,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016HHHT54,0.0,0.0,0.0,0.0,1,0,8,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,15,0,1,1,3,3,17,0.349,2,2,20,71.666667,14.666667,2.666667,0.333333,0.651494,5.666667,0.145030,0.699074,1.000000,1.000000,0.451370,0.666667,...,1,4,7,0,0,4,23,17,0,1,2,80,21,0,17,1,0,2,0,13,0,291,0,4.800,16,3,2,0,48,0,2,2,0,0,3.190,10,0,1,4,108
2016HHHT64,0.0,0.0,0.0,0.0,2,0,7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,7,0,1,0,6,2,13,0.154,2,1,8,34.333333,7.000000,0.666667,0.000000,0.310811,3.333333,0.085470,0.407407,0.333333,0.666667,0.227511,0.666667,...,0,5,2,1,0,3,8,9,0,0,0,39,9,0,7,0,0,0,0,2,1,134,1,0.375,4,1,1,0,27,0,1,2,0,0,1.111,8,0,0,4,64
2016HHKT50,0.0,0.0,0.0,0.0,2,0,5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,14,0,0,0,6,4,13,0.575,0,3,17,76.666667,18.333333,2.333333,0.666667,0.730769,6.333333,0.149606,0.866667,0.000000,0.333333,0.602709,0.666667,...,2,9,10,2,0,5,26,14,0,1,0,82,25,1,11,1,0,1,0,8,1,285,2,2.625,11,3,1,1,52,0,1,0,0,0,2.750,17,0,1,6,100
2016HHLG47,0.0,0.0,0.0,0.0,2,0,11,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,7,0,0,2,13,5,12,0.829,1,4,11,64.666667,11.666667,2.666667,0.333333,0.540871,5.666667,0.155405,1.133333,0.666667,3.000000,0.461584,1.000000,...,1,4,8,3,0,4,22,9,1,3,1,78,18,0,11,1,0,2,3,8,2,287,0,6.000,16,0,2,0,54,0,2,1,1,2,2.929,16,0,0,3,94
2016HHLG55,0.0,0.0,0.0,0.0,0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,7,0,0,0,3,0,4,0.000,2,0,6,28.666667,4.000000,1.000000,0.000000,0.208333,4.666667,0.127273,0.933333,0.666667,0.666667,0.222778,1.333333,...,0,3,1,0,1,3,6,6,0,1,0,35,11,0,4,0,0,1,0,2,0,126,0,1.500,5,0,2,0,24,0,0,0,0,0,1.250,4,0,1,3,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019WOOB30,0.0,0.0,2.0,0.0,2,0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0,9,0,0,0,2,1,10,0.167,1,0,13,66.000000,11.333333,3.000000,1.333333,0.575053,4.666667,0.127985,0.868421,0.000000,1.666667,0.457442,1.000000,...,1,1,6,1,1,5,19,14,1,2,0,77,19,0,9,0,1,0,4,4,2,252,0,3.000,9,0,4,0,54,0,0,0,0,1,1.656,15,0,0,3,60
2019WOOB39,0.0,0.0,2.0,0.0,6,1,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0,16,0,1,0,17,9,22,0.812,3,3,5,65.000000,10.000000,3.333333,1.000000,0.535714,5.333333,0.147024,0.950000,0.000000,1.000000,0.440368,1.333333,...,1,4,2,1,0,3,13,10,1,1,0,68,10,2,15,2,0,1,1,6,1,209,0,2.333,11,0,3,1,54,0,2,1,2,0,2.148,8,0,2,3,78
2019WOSK38,0.0,0.0,2.0,0.0,3,0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0,19,0,1,0,6,5,16,0.567,0,1,16,67.666667,11.666667,2.000000,0.333333,0.556140,4.666667,0.128950,0.597826,1.333333,0.666667,0.425448,0.000000,...,3,4,3,2,0,5,15,12,0,0,0,74,14,0,12,1,0,2,2,4,1,269,0,3.000,17,0,2,1,51,0,0,0,1,0,2.958,6,0,4,7,76


In [ ]:
encoder_list1 = []
match_up_key = 0
for idx in range(0, len(decoder_outputs), 1):
  row = decoder_outputs.iloc[idx]
  if match_up_key == 0:
    match_up_key = row['매치업키']
    previous = vs_r_team[(vs_r_team['일자'] < row['일자']) & (vs_r_team['팀코드'] == row['팀코드'] ) & (vs_r_team['상대팀코드'] == row['상대팀코드']) & (vs_r_team['시즌'] == row['시즌'])].tail(10)
    if len(encoder_list1) == 0:
      encoder_list1 = previous
    else:
      encoder_list1 = pd.concat([encoder_list1, previous])
  elif match_up_key != row['매치업키']:
    match_up_key = row['매치업키']
    previous = vs_r_team[(vs_r_team['일자'] < row['일자']) & (vs_r_team['팀코드'] == row['팀코드'] ) & (vs_r_team['상대팀코드'] == row['상대팀코드']) & (vs_r_team['시즌'] == row['시즌'])].tail(10)
    if len(encoder_list1) == 0:
      encoder_list1 = previous
    else:
      encoder_list1 = pd.concat([encoder_list1, previous])
encoder_list1

,2017,2018,2019,2020,2루타_x,3루타_x,4구_x,HH,HT,KT,LG,LT,OB,SK,SS,WO,게임키,고4_x,단타_x,더블헤더코드,도루_x,도루실패_x,득점,득점권안타,득점권타수,득점권타율,병살타_x,사구_x,삼진_x,상대최근AB,상대최근B1B,상대최근B2B,상대최근B3B,상대최근BABIP,상대최근BB,상대최근BB%_x,상대최근BB/K,상대최근CS,상대최근GDP,상대최근GPA,...,2경기전보크,2경기전병살타_y,2경기전4구_y,3경기전득점권안타,3경기전홈런_x,3경기전희비_x,3경기전4구_x,3경기전안타_x,3경기전잔루,3경기전사구_x,3경기전병살타_x,3경기전도루실패_x,3경기전타자_x,3경기전득점권타수,3경기전초말,3경기전삼진_x,3경기전3루타_x,3경기전고4_x,3경기전실책_x,3경기전도루_x,3경기전자책점,3경기전결과,3경기전투구수,3경기전실책_y,3경기전득점권WHIP,3경기전단타_y,3경기전홈런_y,3경기전2루타_y,3경기전희비_y,3경기전이닝*3,3경기전3루타_y,3경기전사구_y,3경기전도루실패_y,3경기전홀드,3경기전폭투,3경기전2점차이하WHIP,3경기전삼진_y,3경기전보크,3경기전병살타_y,3경기전4구_y
28,0.0,0.0,0.0,0.0,0,0,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,20160405SKLT0,0,5,0,0,0,2,1,9,0.111,0,1,7,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0
36,0.0,0.0,0.0,0.0,3,0,2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,20160406SKLT0,0,8,0,1,1,11,5,10,0.500,0,1,2,30.000000,5.000000,0.000000,0.0,0.217391,3.000000,0.088235,0.428571,0.000000,0.000000,0.160784,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0
46,0.0,0.0,0.0,0.0,1,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,20160407SKLT0,0,3,0,0,0,3,0,3,0.000,0,0,8,28.500000,6.500000,1.500000,0.0,0.355556,2.500000,0.076923,0.555556,0.500000,0.000000,0.318777,...,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0,0,0,0,0,0,0,0.000,0,0,0,0
339,0.0,0.0,0.0,0.0,1,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,20160517LTSK0,0,10,0,0,0,3,2,10,0.200,1,0,8,29.333333,5.333333,1.333333,0.0,0.294118,2.000000,0.061224,0.352941,0.333333,0.000000,0.262090,...,0,1,4,1,0,0,3,5,8,1,0,0,34,9,1,7,0,0,0,0,1,1,152,1,0.750,2,0,2,1,27,0,2,0,3,0,0.667,12,0,0,2
349,0.0,0.0,0.0,0.0,1,1,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,20160518LTSK0,0,5,0,1,1,3,2,11,0.182,0,0,8,31.333333,7.000000,1.666667,0.0,0.351351,1.333333,0.039216,0.222222,0.333333,0.333333,0.280941,...,0,0,3,5,3,0,2,14,5,1,0,1,31,10,1,2,0,0,1,1,0,1,74,0,1.500,1,0,0,0,15,0,1,0,0,0,1.333,2,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4861,0.0,0.0,1.0,0.0,1,0,6,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,20190612LTLG0,3,6,0,1,0,4,1,9,0.111,1,0,5,36.333333,6.333333,1.666667,0.0,0.266667,3.666667,0.084615,0.666667,0.666667,0.333333,0.239550,...,0,1,2,3,1,0,4,7,7,4,0,2,40,8,0,6,0,1,0,0,5,1,182,0,1.636,10,0,1,1,27,0,0,1,0,1,3.273,9,0,0,7
4871,0.0,0.0,1.0,0.0,2,0,2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,20190613LTLG0,0,5,0,0,0,4,1,6,0.167,1,0,7,37.333333,6.333333,2.000000,0.0,0.263158,3.666667,0.083969,0.823529,0.000000,0.666667,0.235900,...,0,2,6,6,1,1,7,13,10,1,0,0,48,13,0,5,0,0,0,0,2,1,116,0,0.857,5,0,1,0,27,1,0,0,0,0,0.667,4,0,1,2
5389,0.0,0.0,1.0,0.0,1,0,2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,20190827LGLT0,1,6,0,0,0,2,0,9,0.000,1,1,11,36.333333,5.333333,1.333333,0.0,0.222222,2.000000,0.049180,0.473684,0.000000,1.000000,0.192403,...,0,0,5,1,0,0,1,6,6,1,1,0,43,4,1,7,0,0,1,0,1,0,194,0,0.857,7,0,1,0,36,1,0,0,1,0,1.250,11,0,2,6
5399,0.0,0.0,1.0,0.0,3,0,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,20190828LGLT0,1,6,0,0,0,3,3,8,0.375,3,0,9,35.000000,5.666667,1.333333,0.0,0.256098,2.000000,0.051282,0.434783,0.000000,1.000000,0.220448,...,0,2,4,1,1,1,6,8,10,0,1,0,43,9,1,5,0,3,0,1,3,1,149,1,1.286,5,0,0,1,30,0,0,0,0,1,0.947,8,0,0,5


In [ ]:
encoder_list2 = []
encoder_list3 = []
for idx in range(0, len(decoder_outputs), 1):
  row = decoder_outputs.iloc[idx]
  if match_up_key == 0:
    match_up_key = row['매치업키']
    previous2 = vs_r_team[(vs_r_team['일자'] < row['일자']) & (vs_r_team['팀코드'] == row['팀코드'] )  & (vs_r_team['시즌'] == row['시즌'])].tail(20)
    previous3 = vs_r_team[(vs_r_team['일자'] < row['일자']) & (vs_r_team['상대팀코드'] == row['상대팀코드']) & (vs_r_team['시즌'] == row['시즌'])].tail(20)
    if len(encoder_list2) == 0:
      encoder_list2 = previous2
      encoder_list3 = previous3
    else:
      encoder_list2 = pd.concat([encoder_list2, previous2])
      encoder_list3 = pd.concat([encoder_list3, previous3])
  elif match_up_key != row['매치업키']:
    match_up_key = row['매치업키']
    previous2 = vs_r_team[(vs_r_team['일자'] < row['일자']) & (vs_r_team['팀코드'] == row['팀코드'] )  & (vs_r_team['시즌'] == row['시즌'])].tail(20)
    previous3 = vs_r_team[(vs_r_team['일자'] < row['일자']) & (vs_r_team['상대팀코드'] == row['팀코드']) & (vs_r_team['시즌'] == row['시즌'])].tail(20)
    if len(encoder_list2) == 0:
      encoder_list2 = previous2
      encoder_list3 = previous3
    else:
      encoder_list2 = pd.concat([encoder_list2, previous2])
      encoder_list3 = pd.concat([encoder_list3, previous3])

In [ ]:
encoder_list3

,2017,2018,2019,2020,2루타_x,3루타_x,4구_x,HH,HT,KT,LG,LT,OB,SK,SS,WO,게임키,고4_x,단타_x,더블헤더코드,도루_x,도루실패_x,득점,득점권안타,득점권타수,득점권타율,병살타_x,사구_x,삼진_x,상대최근AB,상대최근B1B,상대최근B2B,상대최근B3B,상대최근BABIP,상대최근BB,상대최근BB%_x,상대최근BB/K,상대최근CS,상대최근GDP,상대최근GPA,...,2경기전보크,2경기전병살타_y,2경기전4구_y,3경기전득점권안타,3경기전홈런_x,3경기전희비_x,3경기전4구_x,3경기전안타_x,3경기전잔루,3경기전사구_x,3경기전병살타_x,3경기전도루실패_x,3경기전타자_x,3경기전득점권타수,3경기전초말,3경기전삼진_x,3경기전3루타_x,3경기전고4_x,3경기전실책_x,3경기전도루_x,3경기전자책점,3경기전결과,3경기전투구수,3경기전실책_y,3경기전득점권WHIP,3경기전단타_y,3경기전홈런_y,3경기전2루타_y,3경기전희비_y,3경기전이닝*3,3경기전3루타_y,3경기전사구_y,3경기전도루실패_y,3경기전홀드,3경기전폭투,3경기전2점차이하WHIP,3경기전삼진_y,3경기전보크,3경기전병살타_y,3경기전4구_y
401,0.0,0.0,0.0,0.0,0,0,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,20160525LGLT0,0,7,0,1,2,1,0,3,0.000,1,0,7,36.000000,8.666667,2.000000,0.333333,0.423077,4.666667,0.112000,0.548387,0.333333,0.333333,0.284875,...,0,3,3,6,0,2,11,17,14,0,1,1,55,18,1,10,1,3,1,1,11,1,201,0,8.000,11,1,3,0,30,1,0,1,0,1,1.565,10,0,0,2
411,0.0,0.0,0.0,0.0,1,0,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,20160526LGLT0,0,5,0,2,1,4,1,7,0.143,0,0,3,32.333333,6.666667,1.000000,0.000000,0.333333,3.333333,0.092593,0.357143,0.666667,0.333333,0.218986,...,0,0,4,3,1,0,6,9,9,0,0,0,38,10,1,11,0,0,1,1,3,1,146,0,0.429,7,1,0,0,27,0,1,0,1,0,0.818,8,0,3,3
416,0.0,0.0,0.0,0.0,4,0,5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20160527LTHH0,1,12,0,2,0,11,7,19,0.368,0,2,9,35.666667,8.333333,1.333333,0.000000,0.353659,5.000000,0.118110,0.666667,0.666667,0.333333,0.266359,...,0,2,7,1,1,0,6,8,11,0,0,0,44,5,0,11,0,0,1,0,3,0,168,0,2.000,1,0,3,1,29,1,1,1,2,0,1.345,8,0,0,8
426,0.0,0.0,0.0,0.0,5,0,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20160528LTHH0,1,6,0,0,0,9,6,16,0.375,0,0,6,36.000000,10.000000,2.666667,0.000000,0.441860,4.333333,0.100775,0.681818,0.666667,0.333333,0.315741,...,0,1,4,3,1,0,2,10,7,0,1,0,38,8,0,3,0,0,1,0,8,0,193,0,1.929,13,1,2,2,24,1,1,0,0,0,3.000,6,0,2,7
434,0.0,0.0,0.0,0.0,2,0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20160529LTHH0,0,6,0,1,0,9,4,5,0.800,1,1,6,36.000000,9.333333,4.000000,0.000000,0.481928,4.000000,0.093023,0.600000,0.666667,0.000000,0.336772,...,0,2,4,5,0,1,8,13,10,0,0,2,46,15,0,10,0,1,0,2,5,1,157,1,1.800,9,0,0,1,27,0,0,1,2,0,2.062,9,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5588,0.0,0.0,1.0,0.0,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20190924SSLG0,0,5,0,1,0,4,1,7,0.143,0,0,12,33.333333,5.666667,2.333333,0.000000,0.292683,3.333333,0.085470,0.526316,0.000000,0.000000,0.233553,...,0,1,4,5,0,1,4,10,8,1,0,0,39,11,1,6,0,0,0,0,2,1,136,0,0.750,7,1,0,0,27,0,2,0,1,0,1.333,7,0,3,3
5598,0.0,0.0,1.0,0.0,2,0,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,20190926LGKT0,0,4,0,1,1,3,1,8,0.125,0,0,7,33.333333,5.000000,1.000000,0.000000,0.225000,2.666667,0.072727,0.470588,0.000000,0.666667,0.209318,...,0,1,6,2,2,0,5,11,8,0,0,0,42,12,0,4,0,0,0,1,3,1,139,0,1.286,8,1,1,0,27,0,0,0,0,0,2.000,6,0,0,3
5605,0.0,0.0,1.0,0.0,1,0,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20190927NCLG0,0,7,0,1,2,3,3,12,0.250,0,3,11,31.000000,6.333333,0.000000,0.333333,0.277778,3.000000,0.079646,0.500000,0.666667,0.333333,0.249506,...,0,1,3,2,1,1,4,9,8,1,0,1,40,4,0,10,1,0,2,1,0,1,134,0,1.500,3,0,0,0,27,0,2,0,2,1,0.600,9,0,0,1
5609,0.0,0.0,1.0,0.0,1,0,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20190928LGHT0,0,12,0,1,0,7,4,15,0.267,1,2,9,33.666667,6.333333,0.666667,0.000000,0.247059,2.333333,0.063636,0.437500,0.333333,0.666667,0.189559,...,0,1,2,2,0,0,4,7,9,0,0,0,39,8,0,5,0,0,1,0,13,0,209,1,2.357,12,0,1,1,24,1,3,0,0,1,12.000,9,0,0,7


In [ ]:
decoder_outputs.to_csv("./gdrive/My Drive/빅콘테스트2020/EDA결과/모델/사생결단_decoder_output.csv", index=False, encoding='utf-8-sig')
encoder_list1.to_csv("./gdrive/My Drive/빅콘테스트2020/EDA결과/모델/사생결단_encoder_input_vs_r.csv", index=False, encoding='utf-8-sig')
encoder_list2.to_csv("./gdrive/My Drive/빅콘테스트2020/EDA결과/모델/사생결단_encoder_input_h.csv", index=False, encoding='utf-8-sig')
encoder_list3.to_csv("./gdrive/My Drive/빅콘테스트2020/EDA결과/모델/사생결단_encoder_input_a.csv", index=False, encoding='utf-8-sig')

In [ ]:
decoder_outputs = pd.read_csv("./gdrive/My Drive/빅콘테스트2020/EDA결과/모델/사생결단_decoder_output.csv", encoding='utf-8-sig')
encoder_list1 = pd.read_csv("./gdrive/My Drive/빅콘테스트2020/EDA결과/모델/사생결단_encoder_input_vs_r.csv", encoding='utf-8-sig')
encoder_list2 = pd.read_csv("./gdrive/My Drive/빅콘테스트2020/EDA결과/모델/사생결단_encoder_input_h.csv", encoding='utf-8-sig')
encoder_list3 = pd.read_csv("./gdrive/My Drive/빅콘테스트2020/EDA결과/모델/사생결단_encoder_input_a.csv", encoding='utf-8-sig')

In [ ]:
decoder_outputs = decoder_outputs[decoder_outputs['초말'] == 1]

In [ ]:
lstm_inputs = lstm_inputs[["득점권안타", "안타_y" , "득점권WHIP" , "안타_x", "득점권타율", "홈런_x", "타자_x", "타수_y", "타자_y", "타수_x" , "홈런_y" , "득점권타수", "희비_x", "잔루", "희타_x", "4구_x", "희비_y", "2루타_x", "삼진_y" , "단타_x", "삼진_x" , "4구_y" , "고4_y" , "실책_x", "도루_x", 'OB', 'WO', 'HT', "LG", 'SS', 'KT', 'LT', 'SK', "HH",'상대팀OB', '상대팀WO', '상대팀HT', "상대팀LG", '상대팀SS', '상대팀KT', '상대팀LT', '상대팀SK', "상대팀HH", "도루_y", "폭투", "사구_y", "사구_x", "3루타_x", "3루타_y", "병살타_y", "병살타_x", "실책_y", "보크", "도루실패_y", "더블헤더코드", "도루실패_x", '2017', '2018', '2019', '2020', '시즌']]
decoder_outputs = decoder_outputs[['결과',"시즌"]]
encoder_list1 = encoder_list1[["득점권안타", "안타_y" , "득점권WHIP" , "안타_x", "득점권타율", "홈런_x", "타자_x", "타수_y", "타자_y", "타수_x" , "홈런_y" , "득점권타수", "희비_x", "잔루", "희타_x", "4구_x", "희비_y", "2루타_x", "삼진_y" , "단타_x", "삼진_x" , "4구_y" , "고4_y" , "실책_x", "도루_x", 'OB', 'WO', 'HT', "LG", 'SS', 'KT', 'LT', 'SK', "HH",'상대팀OB', '상대팀WO', '상대팀HT', "상대팀LG", '상대팀SS', '상대팀KT', '상대팀LT', '상대팀SK', "상대팀HH", "도루_y", "폭투", "사구_y", "사구_x", "3루타_x", "3루타_y", "병살타_y", "병살타_x", "실책_y", "보크", "도루실패_y", "더블헤더코드", "도루실패_x", '2017', '2018', '2019', '2020','시즌']]
encoder_list2 = encoder_list2[["득점권안타", "안타_y" , "득점권WHIP" , "안타_x", "득점권타율", "홈런_x", "타자_x", "타수_y", "타자_y", "타수_x" , "홈런_y" , "득점권타수", "희비_x", "잔루", "희타_x", "4구_x", "희비_y", "2루타_x", "삼진_y" , "단타_x", "삼진_x" , "4구_y" , "고4_y" , "실책_x", "도루_x", 'OB', 'WO', 'HT', "LG", 'SS', 'KT', 'LT', 'SK', "HH",'상대팀OB', '상대팀WO', '상대팀HT', "상대팀LG", '상대팀SS', '상대팀KT', '상대팀LT', '상대팀SK', "상대팀HH", "도루_y", "폭투", "사구_y", "사구_x", "3루타_x", "3루타_y", "병살타_y", "병살타_x", "실책_y", "보크", "도루실패_y", "더블헤더코드", "도루실패_x", '2017', '2018', '2019', '2020','시즌']]

In [ ]:
decoder_outputs

,2017,2018,2019,2020,2루타_x,3루타_x,4구_x,HH,HT,KT,LG,LT,OB,SK,SS,WO,게임키,고4_x,단타_x,더블헤더코드,도루_x,도루실패_x,득점,득점권안타,득점권타수,득점권타율,병살타_x,사구_x,삼진_x,상대최근AB,상대최근B1B,상대최근B2B,상대최근B3B,상대최근BABIP,상대최근BB,상대최근BB%_x,상대최근BB/K,상대최근CS,상대최근GDP,상대최근GPA,...,2경기전4구_y,3경기전득점권안타,3경기전홈런_x,3경기전희비_x,3경기전4구_x,3경기전안타_x,3경기전잔루,3경기전사구_x,3경기전병살타_x,3경기전도루실패_x,3경기전타자_x,3경기전득점권타수,3경기전초말,3경기전삼진_x,3경기전3루타_x,3경기전고4_x,3경기전실책_x,3경기전도루_x,3경기전자책점,3경기전결과,3경기전투구수,3경기전실책_y,3경기전득점권WHIP,3경기전단타_y,3경기전홈런_y,3경기전2루타_y,3경기전희비_y,3경기전이닝*3,3경기전3루타_y,3경기전사구_y,3경기전도루실패_y,3경기전홀드,3경기전폭투,3경기전2점차이하WHIP,3경기전삼진_y,3경기전보크,3경기전병살타_y,3경기전4구_y,매치업,매치업키
0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,20160618SKLT0,0,8,0,0,0,0,0,4,0.000,2,0,4,31.666667,5.333333,1.000000,0.000000,0.256757,4.000000,0.109091,0.600000,0.000000,1.000000,0.216447,...,3,2,1,0,4,7,7,0,1,0,37,12,0,6,0,0,0,1,2,1,179,0,2.250,6,0,0,1,27,0,0,0,1,0,1.105,10,0,2,5,23,2016LTSK23
1,0.0,0.0,0.0,0.0,7,0,5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,20160619SKLT0,0,3,0,1,0,10,5,14,0.357,1,2,5,32.000000,6.666667,0.333333,0.000000,0.269231,2.666667,0.075472,0.444444,0.000000,1.333333,0.196280,...,3,2,1,1,7,7,6,0,2,0,38,9,0,6,0,0,0,4,6,0,124,0,0.900,4,1,2,0,24,1,1,0,0,0,1.375,7,0,1,3,23,2016LTSK23
2,0.0,0.0,0.0,0.0,4,0,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,20160623KTOB0,0,11,0,0,0,4,2,12,0.167,2,0,2,36.333333,9.000000,1.666667,0.000000,0.340426,2.666667,0.065041,0.571429,0.000000,0.333333,0.279741,...,4,2,0,1,2,8,5,0,0,0,36,6,0,3,0,0,0,0,4,0,136,1,1.000,10,1,0,1,24,0,0,1,0,1,1.800,9,0,1,0,24,2016OBKT24
3,0.0,0.0,0.0,0.0,2,0,5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,20160625OBSK0,1,6,0,0,0,6,3,12,0.250,0,0,8,32.000000,5.333333,1.000000,0.666667,0.276316,0.666667,0.019802,0.117647,0.666667,0.666667,0.240583,...,5,1,3,1,1,9,3,0,1,0,36,4,0,8,1,0,1,0,3,0,131,0,2.625,6,0,3,1,24,0,1,0,0,3,3.600,5,0,0,3,25,2016SKOB25
4,0.0,0.0,0.0,0.0,3,0,3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,20160626OBSK0,0,13,0,0,2,6,4,10,0.400,1,2,8,33.000000,6.000000,1.333333,0.333333,0.291139,1.666667,0.047170,0.352941,0.666667,0.333333,0.243630,...,1,0,0,0,0,7,4,0,1,2,31,1,0,3,0,0,1,0,4,0,152,2,1.091,6,1,2,1,24,0,0,0,0,1,3.000,5,0,0,5,25,2016SKOB25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011,0.0,0.0,1.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20190927SKSS0,0,4,0,3,0,0,0,5,0.000,1,0,7,29.000000,4.666667,1.000000,0.000000,0.236111,2.666667,0.080000,0.470588,1.000000,0.333333,0.176859,...,3,3,0,2,6,7,8,1,0,1,37,13,1,0,0,0,1,2,3,1,177,1,1.091,11,0,0,0,27,0,0,0,1,0,1.091,7,0,0,5,64,2019SSSK64
1012,0.0,0.0,1.0,0.0,5,0,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20190928SKSS0,1,6,0,1,0,9,5,15,0.333,0,0,10,30.000000,4.333333,0.333333,0.000000,0.212121,0.666667,0.021505,0.083333,0.666667,0.666667,0.119928,...,2,0,0,0,0,5,4,0,0,1,31,1,1,7,0,0,0,0,2,0,128,0,0.667,3,0,2,0,27,0,0,0,0,0,0.889,9,0,0,3,64,2019SSSK64
1013,0.0,0.0,1.0,0.0,1,1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,20191001NCOB0,0,13,0,0,0,6,4,15,0.267,0,0,6,36.666667,5.000000,0.333333,0.333333,0.184783,1.333333,0.034783,0.266667,0.000000,0.000000,0.179644,...,2,0,2,0,0,4,2,0,0,0,31,0,0,3,0,0,0,0,3,0,109,0,3.000,5,1,2,0,24,0,0,0,0,0,1.304,2,0,1,2,65,2019OBNC65
1014,0.0,0.0,1.0,0.0,0,0,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,20190929OBLG0,0,4,0,0,0,0,0,1,0.000,0,0,6,36.333333,8.000000,2.000000,0.333333,0.373494,2.000000,0.051282,0.260870,0.333333,0.666667,0.285199,...,4,0,1,0,2,8,8,0,1,0,34,7,1,6,0,0,1,0,1,1,152,0,0.429,7,0,1,1,27,0,0,1,3,0,1.222,5,0,0,3,66,2019LGOB66


In [ ]:
decoder_input = decoder_outputs.groupby(by='매치업키')
decoder_input.head(1)[['팀코드', '상대팀코드', '안타_x']]

,팀코드,상대팀코드,안타_x
0,LT,SK,8
2,OB,KT,15
3,SK,OB,10
5,HT,LG,9
6,WO,HH,16
...,...,...,...
1010,LG,NC,7
1011,SS,SK,4
1013,OB,NC,15
1014,LG,OB,4


In [ ]:
vs_r_mean = []
match_up_key = 0
for idx in range(0, len(decoder_outputs), 1):
  row = decoder_outputs.iloc[idx]
  if match_up_key == 0:
    match_up_key = row['매치업키']
    previous = vs_r_team[(vs_r_team['일자'] < row['일자']) & (vs_r_team['팀코드'] == row['팀코드'] ) & (vs_r_team['상대팀코드'] == row['상대팀코드']) & (vs_r_team['시즌'] == row['시즌'])].tail(3).groupby('팀코드').mean()
    previous['매치업키'] = row['매치업키']
    if len(vs_r_mean) == 0:
      vs_r_mean = previous
    else:
      vs_r_mean = pd.concat([vs_r_mean, previous])
  elif match_up_key != row['매치업키']:
    match_up_key = row['매치업키']
    previous = vs_r_team[(vs_r_team['일자'] < row['일자']) & (vs_r_team['팀코드'] == row['팀코드'] ) & (vs_r_team['상대팀코드'] == row['상대팀코드']) & (vs_r_team['시즌'] == row['시즌'])].tail(3).groupby('팀코드').mean()
    previous['매치업키'] = row['매치업키']
    if len(vs_r_mean) == 0:
      vs_r_mean = previous
    else:
      vs_r_mean = pd.concat([vs_r_mean, previous])
vs_r_mean

,2017,2018,2019,2020,2루타_x,3루타_x,4구_x,HH,HT,KT,LG,LT,OB,SK,SS,WO,고4_x,단타_x,더블헤더코드,도루_x,도루실패_x,득점,득점권안타,득점권타수,득점권타율,병살타_x,사구_x,삼진_x,상대최근AB,상대최근B1B,상대최근B2B,상대최근B3B,상대최근BABIP,상대최근BB,상대최근BB%_x,상대최근BB/K,상대최근CS,상대최근GDP,상대최근GPA,상대최근HBP,...,2경기전병살타_y,2경기전4구_y,3경기전득점권안타,3경기전홈런_x,3경기전희비_x,3경기전4구_x,3경기전안타_x,3경기전잔루,3경기전사구_x,3경기전병살타_x,3경기전도루실패_x,3경기전타자_x,3경기전득점권타수,3경기전초말,3경기전삼진_x,3경기전3루타_x,3경기전고4_x,3경기전실책_x,3경기전도루_x,3경기전자책점,3경기전결과,3경기전투구수,3경기전실책_y,3경기전득점권WHIP,3경기전단타_y,3경기전홈런_y,3경기전2루타_y,3경기전희비_y,3경기전이닝*3,3경기전3루타_y,3경기전사구_y,3경기전도루실패_y,3경기전홀드,3경기전폭투,3경기전2점차이하WHIP,3경기전삼진_y,3경기전보크,3경기전병살타_y,3경기전4구_y,매치업키
팀코드,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
LT,0.0,0.0,0.0,0.0,1.000000,0.000000,4.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,5.333333,0.0,1.666667,0.000000,3.000000,1.333333,10.000000,0.129667,1.000000,0.000000,6.666667,34.666667,6.333333,1.444444,0.333333,0.302589,6.222222,0.149333,0.904545,0.333333,0.666667,0.271857,0.000000,...,1.333333,3.666667,2.000000,0.666667,0.333333,6.000000,9.333333,9.666667,0.000000,0.333333,0.666667,42.000000,11.333333,0.000000,8.000000,0.666667,0.333333,0.333333,0.333333,4.666667,0.666667,126.000000,0.000000,1.976333,5.333333,2.000000,0.333333,0.333333,26.0,0.333333,0.000000,0.333333,0.333333,0.000000,1.121667,7.666667,0.000000,1.000000,2.333333,2016LTSK23
OB,0.0,0.0,0.0,0.0,1.666667,0.000000,2.666667,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,9.000000,0.0,1.000000,0.000000,7.666667,3.666667,10.333333,0.351333,0.333333,0.666667,4.666667,36.111111,7.111111,2.555556,0.444444,0.326791,4.222222,0.101292,0.833333,0.000000,0.888889,0.286141,0.111111,...,1.333333,2.333333,5.000000,1.333333,1.000000,6.333333,12.000000,8.333333,0.000000,1.666667,0.000000,42.666667,12.666667,0.666667,5.333333,0.666667,0.333333,0.000000,1.333333,4.666667,1.000000,154.333333,0.000000,1.343000,6.666667,0.666667,2.666667,0.000000,27.0,0.000000,1.000000,0.000000,0.333333,0.333333,1.333333,7.666667,0.000000,1.666667,4.000000,2016OBKT24
SK,0.0,0.0,0.0,0.0,1.000000,0.666667,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,5.333333,0.0,0.333333,0.666667,3.666667,1.000000,3.333333,0.216667,0.666667,0.333333,5.666667,31.555556,5.000000,0.777778,0.222222,0.246966,1.555556,0.045205,0.241865,0.555556,1.000000,0.223713,0.555556,...,0.666667,3.000000,1.333333,1.000000,0.333333,2.000000,7.666667,5.666667,0.666667,1.000000,0.333333,34.666667,5.666667,0.666667,6.666667,0.000000,0.000000,0.333333,0.000000,4.333333,0.333333,144.333333,0.000000,1.202000,7.000000,1.333333,2.000000,0.666667,26.0,0.000000,0.000000,0.666667,0.333333,0.333333,1.787333,5.666667,0.333333,1.333333,3.666667,2016SKOB25
HT,0.0,0.0,0.0,0.0,1.666667,0.000000,3.666667,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,8.000000,0.0,0.333333,0.666667,8.333333,3.666667,10.000000,0.407333,0.333333,0.666667,5.333333,34.444444,7.333333,0.666667,0.000000,0.289732,2.000000,0.051826,0.325851,0.111111,0.666667,0.232660,0.888889,...,0.333333,3.666667,1.333333,0.333333,0.666667,1.666667,7.333333,6.333333,0.666667,1.000000,0.000000,36.000000,7.000000,0.000000,6.666667,0.000000,0.000000,1.000000,0.666667,5.666667,0.333333,148.333333,0.000000,1.833333,8.000000,0.000000,1.333333,1.000000,25.0,0.666667,0.000000,0.000000,0.666667,1.666667,2.313667,7.000000,0.000000,0.333333,4.333333,2016HTLG26
WO,0.0,0.0,0.0,0.0,2.333333,0.000000,5.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,8.666667,0.0,2.333333,0.000000,8.333333,4.000000,13.666667,0.278667,0.666667,0.333333,7.000000,35.222222,6.777778,1.444444,0.222222,0.318974,5.666667,0.135404,0.728981,0.333333,0.777778,0.272437,0.111111,...,0.666667,4.000000,3.333333,1.000000,0.333333,6.000000,9.333333,7.666667,0.333333,1.000000,0.666667,41.000000,10.000000,0.666667,7.333333,0.333333,0.333333,0.333333,0.666667,7.666667,0.333333,159.000000,0.333333,1.181667,9.000000,1.666667,2.000000,0.666667,26.0,0.000000,0.333333,1.000000,0.333333,1.000000,1.999000,5.666667,0.000000,0.666667,4.000000,2016WOHH26
...,...,...,...,.

In [ ]:
vs_r_mean = vs_r_mean[['매치업키', '안타_x']]

In [ ]:
decoder_outputs = decoder_outputs[['팀코드', '매치업키']]

In [ ]:
list_1 = np.array([[[]]])
for idx in range(0, len(vs_r_mean)):
  row = vs_r_mean.iloc[idx]
  list_2 = np.array([row.values])
  row_raw = decoder_outputs[decoder_outputs['매치업키'] == row['매치업키']]
  for j in range(len(row_raw) - 1):
    row2 = row_raw.iloc[j]
    list_2 = np.append(list_2,[row2.values])
  # if len(list_2) == 1:
  #   list_2.append([])
  print(list_2)

['2016LTSK23' 7.0 'LT' '2016LTSK23']
[['2016OBKT24' 11.666666666666666]]
['2016SKOB25' 8.333333333333334 'SK' '2016SKOB25']
[['2016HTLG26' 11.333333333333334]]
['2016WOHH26' 11.666666666666666 'WO' '2016WOHH26']
['2016HHSS29' 10.333333333333334 'HH' '2016HHSS29']
['2016KTWO29' 8.333333333333334 'KT' '2016KTWO29']
['2016NCOB29' 7.0 'NC' '2016NCOB29']
['2016OBHT29' 12.333333333333334 'OB' '2016OBHT29']
[['2016SKKT29' 10.333333333333334]]
['2016SSLT29' 8.666666666666666 'SS' '2016SSLT29']
[['2016NCSK30' 11.333333333333334]]
['2016KTSS31' 14.0 'KT' '2016KTSS31']
['2016LTHH31' 10.0 'LT' '2016LTHH31']
['2016WOLG31' 9.666666666666666 'WO' '2016WOLG31']
[['2016HTNC32' 10.333333333333334]]
[['2016LGLT33' 11.666666666666666]]
[['2016NCLG33' 8.333333333333334]]
[['2016WOOB33' 9.666666666666666]]
['2016SKHT34' 11.333333333333334 'SK' '2016SKHT34']
[['2016HTHH35' 11.666666666666666]]
[['2016SKSS35' 16.333333333333332]]
['2016SSHT35' 13.0 'SS' '2016SSHT35']
['2016KTWO36' 13.333333333333334 'KT' '201

In [ ]:
list_1

array([0.0, 0.0, 0.0, ..., 1.3333333333333333, 5.0, '2019LGLT67'],
      dtype=object)

In [ ]:
list_1 = []
for idx in range(0, len(vs_r_mean)):
  list_2 = []
  row = vs_r_mean.iloc[idx]
  list_2.append([row.values])
  row_raw = decoder_outputs[decoder_outputs['매치업키'] == row['매치업키']]
  for j in range(len(row_raw) - 1):
    list_2.append([row_raw.iloc[j].values])
  if len(list_2) == 1:
    list_2.append([])
  list_1.append(list_2)

In [ ]:
for i in range(len(list_1)):
  list_1[i] = np.concatenate(list_1[i],axis=0)
li = np.concatenate(list_1,axis=0)
li.shape

ValueError: ignored

In [ ]:
np.array(list_1).shape

(626,)

In [ ]:
li = np.array(list_1[0])
li.shape

(2, 1)

In [ ]:
li

array([[array([[0.0],
       [0.0],
       [0.0],
       [0.0],
       [1.0],
       [0.0],
       [4.0],
       [0.0],
       [0.0],
       [0.0],
       [0.0],
       [1.0],
       [0.0],
       [0.0],
       [0.0],
       [0.0],
       [0.0],
       [5.333333333333333],
       [0.0],
       [1.6666666666666667],
       [0.0],
       [3.0],
       [1.3333333333333333],
       [10.0],
       [0.12966666666666668],
       [1.0],
       [0.0],
       [6.666666666666667],
       [34.666666666666664],
       [6.333333333333333],
       [1.4444444444444444],
       [0.3333333333333333],
       [0.3025888093638771],
       [6.222222222222222],
       [0.14933333333333335],
       [0.9045454545454547],
       [0.3333333333333333],
       [0.6666666666666666],
       [0.27185733618540375],
       [0.0],
       [9.111111111111112],
       [1.0],
       [0.024000000000000004],
       [0.3333333333333333],
       [0.14774069129209316],
       [7.333333333333333],
       [0.17600000000000002],
  

In [ ]:
lstm_inputs_train = lstm_inputs[lstm_inputs['시즌'] != 2019]
lstm_inputs_test = lstm_inputs[lstm_inputs['시즌'] == 2019]
decoder_outputs_train = decoder_outputs[decoder_outputs['시즌'] != 2019]
decoder_outputs_test = decoder_outputs[decoder_outputs['시즌'] == 2019]
encoder_list1_train = encoder_list1[encoder_list1['시즌'] != 2019]
encoder_list1_test = encoder_list1[encoder_list1['시즌'] == 2019]
encoder_list2_train = encoder_list2[encoder_list2['시즌'] != 2019]
encoder_list2_test = encoder_list2[encoder_list2['시즌'] == 2019]

In [ ]:
del lstm_inputs_train['시즌']
del lstm_inputs_test['시즌']
del decoder_outputs_train['시즌']
del decoder_outputs_test['시즌']
del encoder_list1_train['시즌']
del encoder_list1_test['시즌']
del encoder_list2_train['시즌']
del encoder_list2_test['시즌']

In [ ]:
lstm_inputs_train = lstm_inputs_train.values 
lstm_inputs_test = lstm_inputs_test.values
decoder_outputs_train = decoder_outputs_train.values
decoder_outputs_test = decoder_outputs_test.values
encoder_list1_train = encoder_list1_train.values
encoder_list1_test = encoder_list1_test.values
encoder_list2_train = encoder_list2_train.values
encoder_list2_test = encoder_list2_test.values

In [ ]:
lstm_inputs_train = lstm_inputs_train.reshape(-1, 10, 60)
lstm_inputs_test = lstm_inputs_test.reshape(-1, 10, 60)
encoder_list1_train = encoder_list1_train.reshape(-1, 20, 60)
encoder_list1_test = encoder_list1_test.reshape(-1, 20, 60)
encoder_list2_train = encoder_list2_train.reshape(-1, 20, 60)
encoder_list2_test = encoder_list2_test.reshape(-1, 20, 60)

In [ ]:
decoder_outputs_train

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [1]])

In [ ]:
lstm_inputs_train[0]

array([[ 1.   ,  4.   ,  0.75 ,  5.   ,  0.111,  0.   , 34.   , 30.   ,
        35.   , 30.   ,  0.   ,  9.   ,  0.   ,  8.   ,  0.   ,  3.   ,
         1.   ,  0.   , 12.   ,  5.   ,  7.   ,  2.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  1.   ,  0.   ,  1.   ,  0.   ,  2.   ,  1.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ],
       [ 5.   ,  1.   ,  1.5  , 14.   ,  0.5  ,  3.   , 31.   , 16.   ,
        21.   , 27.   ,  0.   , 10.   ,  0.   ,  5.   ,  1.   ,  2.   ,
         0.   ,  3.   ,  2.   ,  8.   ,  2.   ,  4.   ,  0.   ,  1.   ,
         1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  1.   ,  1.   ,  0.   ,
         0.   ,  1.   ,

In [ ]:
pip install -U keras-tuner

     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=8c7eb3bd39b9df88d36b43e45c9edd3fecc622b84b44ba90b4effc8bb80a1d29
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=004c2764d01ba64af55601b852d10ebd292b26d02e8e1096957c6ca4e4512b59
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.optimizers import Adam
from keras import backend as K
# returns train, inference_encoder and inference_decoder models
def define_models(hp):
  n_units = hp.Int('total_units', min_value = 4, max_value = 64, step = 4)

  # define training encoder
  encoder_inputs = Input(shape=(10,60))
  encoder = LSTM(n_units, return_state=True,activation='relu')
  encoder_outputs, state_h, state_c = encoder(encoder_inputs)
  encoder_states = [state_h, state_c]
  # define training encoder
  encoder_inputs2 = Input(shape=(20,60))
  encoder2 = LSTM(n_units, return_state=True,activation='relu')
  encoder_outputs2, state_h2, state_c2 = encoder2(encoder_inputs)
  encoder_states2 = [state_h2, state_c2]
  # define training encoder2
  encoder_inputs3 = Input(shape=(20,60))
  encoder3 = LSTM(n_units, return_state=True, activation='relu')
  encoder_outputs3, state_h3, state_c3 = encoder3(encoder_inputs3)
  encoder_states3 = [state_h3, state_c3]

  

   # define training decoder
  decoder_inputs = Input(shape=(None, 60))
  decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
  decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states, activation='relu')
  decoder_timedistribute = TimeDistributed(Dense(hp.Int('units', min_value = 4, max_value = 64, step = 4), activation='relu'))
  decoder_outputs = decoder_timedistribute(decoder_outputs)

  decoder_lstm2 = LSTM(n_units, return_sequences=True, return_state=True)
  decoder_outputs2, _, _ = decoder_lstm2(decoder_outputs, initial_state=encoder_states2, activation='relu')
  decoder_timedistribute2 = TimeDistributed(Dense(hp.Int('units', min_value = 4, max_value = 64, step = 4), activation='relu'))
  decoder_outputs2 = decoder_timedistribute2(decoder_outputs)

  # define training decoder2
  decoder_lstm3 = LSTM(60, return_sequences=True, return_state=True)
  decoder_outputs3, _, _ = decoder_lstm3(decoder_outputs2, initial_state=encoder_states3, activation='relu')
  decoder_timedistribute3 = TimeDistributed(Dense(1, activation='sigmoid'))
  output = decoder_timedistribute3(decoder_outputs3)

  model = Model(inputs=[encoder_inputs, encoder_inputs2, encoder_inputs3, decoder_inputs], outputs=output)
 
   
  model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-3,1e-4,1e-5])),
                  loss = 'binary_crossentropy',
                  metrics=['binary_accuracy'])
   # return all models
  return model

In [ ]:
from kerastuner.tuners import RandomSearch

tuner = RandomSearch(define_models,
                     objective='val_acc',   #모니터링 할 평가지표
                     max_trials=5,          #5개의 모델
                     executions_per_trial=3,#모델당 3번의 학습
                     directory='./gdrive/My Drive/빅콘테스트2020/EDA결과/모델',         #경로지정
                     project_name='EndGame')   #프로젝트명

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-171-c36d65102acd>", line 28, in define_models
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states, activation='relu')
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/layers/recurrent.py", line 709, in __call__
    output = super(RNN, self).__call__(full_input, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 926, in __call__
    input_list)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 1098, in _functional_construction_call
    self._maybe_build(inputs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 2643, in _maybe_build
    self.build(input_shapes)  # pylint:disable=not-call

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-171-c36d65102acd>", line 28, in define_models
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states, activation='relu')
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/layers/recurrent.py", line 709, in __call__
    output = super(RNN, self).__call__(full_input, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 926, in __call__
    input_list)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 1098, in _functional_construction_call
    self._maybe_build(inputs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 2643, in _maybe_build
    self.build(input_shapes)  # pylint:disable=not-call

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-171-c36d65102acd>", line 28, in define_models
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states, activation='relu')
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/layers/recurrent.py", line 709, in __call__
    output = super(RNN, self).__call__(full_input, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 926, in __call__
    input_list)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 1098, in _functional_construction_call
    self._maybe_build(inputs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 2643, in _maybe_build
    self.build(input_shapes)  # pylint:disable=not-call

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-171-c36d65102acd>", line 28, in define_models
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states, activation='relu')
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/layers/recurrent.py", line 709, in __call__
    output = super(RNN, self).__call__(full_input, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 926, in __call__
    input_list)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 1098, in _functional_construction_call
    self._maybe_build(inputs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 2643, in _maybe_build
    self.build(input_shapes)  # pylint:disable=not-call

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-171-c36d65102acd>", line 28, in define_models
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states, activation='relu')
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/layers/recurrent.py", line 709, in __call__
    output = super(RNN, self).__call__(full_input, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 926, in __call__
    input_list)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 1098, in _functional_construction_call
    self._maybe_build(inputs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 2643, in _maybe_build
    self.build(input_shapes)  # pylint:disable=not-call

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/engine/hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-171-c36d65102acd>", line 28, in define_models
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states, activation='relu')
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/layers/recurrent.py", line 709, in __call__
    output = super(RNN, self).__call__(full_input, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 926, in __call__
    input_list)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 1098, in _functional_construction_call
    self._maybe_build(inputs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py", line 2643, in _maybe_build
    self.build(input_shapes)  # pylint:disable=not-call

RuntimeError: ignored

In [ ]:
np.array([ [ [1, 2, 3], [4, 5, 6]], [ [2, 3, 4], [4, 5, 2], [1, 2, 4]]]).shape

(2,)